In [1]:
import pandas as pd
import re
import numpy as np

In [2]:
class args():
    pass
args.model_name = "climatebert/distilroberta-base-climate-f"
args.y_prefix = "10"
y_prefix = args.y_prefix
min_samples = 10
inclusion_var="INCLUDE"

In [3]:
df = pd.read_feather('../data/labels.feather')
df = (df
      .sort_values('id')
      .sample(frac=1, random_state=1)
      .reset_index(drop=True)
)
df['seen']=1


In [6]:
targets = [x for x in df.columns if re.match(f"^{y_prefix}",x)]
targets = [x for x in targets if df[x].sum()>min_samples]
df = df.replace(-1,np.NaN)
df = df.replace(2, 1)
df = df.dropna(subset=targets).reset_index(drop=True)
if y_prefix in inclusion_var:
    nonrandom_index = df[df["representative_sample"]==0].index
    df['labels'] = df[targets[0]]
else:
    df = df[df[inclusion_var]==1].reset_index(drop=True)
    nonrandom_index = df[df["representative_relevant"]==0].index
    df['labels'] = list(df[targets].values.astype(int))

targets


['10 - 3. Quantitative', '10 - 4. Qualitative']

In [4]:
df.columns

Index(['nacsos_id', 'title', 'abstract', '0 - relevant',
       '10 - 3. Quantitative', '10 - 4. Qualitative', '2 - 0. Mitigation',
       '2 - 1. Adaptation', '3 - 0. Not policy related',
       '3 - 1. Analysis of an existing policy / concrete proposal',
       ...
       '5 - 4.17. Government administration & management',
       'representative_sample', 'representative_relevant',
       '17 - 0. Supranational and international', '17 - 1. National',
       '17 - 2. Sub-national', '19 - 0. Ex-post', '19 - 1. Ex-ante', 'id',
       'seen'],
      dtype='object', length=110)

In [7]:
y_pred = np.load("cv_results/bert-tiny__19 -__1__outer_predictions.npy")
y_pred

array([[0.04878383, 0.94671875],
       [0.10026041, 0.93964493],
       [0.88013387, 0.46886945],
       [0.87324774, 0.49748972],
       [0.9522103 , 0.06780828],
       [0.2017804 , 0.9204664 ],
       [0.9539371 , 0.081968  ],
       [0.5123893 , 0.8424988 ],
       [0.05071485, 0.9505645 ],
       [0.9490737 , 0.07786987],
       [0.04841587, 0.95061326],
       [0.20563701, 0.9146551 ],
       [0.06344476, 0.90464824],
       [0.43779713, 0.8575195 ],
       [0.04716782, 0.9484135 ],
       [0.04669519, 0.95064443],
       [0.89143884, 0.25497139],
       [0.28399476, 0.87579966],
       [0.9291732 , 0.09779079],
       [0.05733116, 0.9488737 ],
       [0.7408345 , 0.70751786],
       [0.04471415, 0.94614255],
       [0.05353111, 0.95037484],
       [0.95169383, 0.08496623],
       [0.04830634, 0.9492611 ],
       [0.04773958, 0.94849885],
       [0.94778293, 0.06458968],
       [0.94711417, 0.13433097],
       [0.95143455, 0.071148  ],
       [0.13258663, 0.9123046 ],
       [0.

In [26]:
df[targets].sum(axis=1).unique()

array([1., 2., 0.])

In [4]:

args.y_prefix = "4 - "
args.y_prefix = "INCLUDE"
#args.y_prefix = "8 - "
args.n_splits = 3
args.make_predictions = "False"
args.roundup=False
cols = [x for x in seen_df.columns if re.match(f"^{args.y_prefix}",x)]


# If the target is inclusion, use only those documents for which we have a non-na value
# Otherwise, only use those documents which are included
# also define what subset is to be treated as a random representative sample
# For labels beyond inclusion, we treat all those that are representative of the included
# studies as representative
if "INCLUDE" in args.y_prefix:
    y_var = cols[0]
    seen_df = seen_df.loc[pd.notna(seen_df[y_var]),:].reset_index(drop=True)
    seen_df['random'] = seen_df['representative_sample']
else:
    seen_df = seen_df[seen_df['INCLUDE']==1]
    seen_df['random'] = seen_df['representative_relevant']

# Turn the columns into target variables and get class-weights to counteract class imbalances
if len(cols)==1:
    y_var = cols[0]
    seen_df = seen_df.loc[pd.notna(seen_df[y_var]),:].reset_index(drop=True)
    print(seen_df.shape)
    seen_df['labels'] = list(seen_df[y_var].values.astype(int))
    cw = seen_df[(seen_df['random']==1) & (seen_df[y_var]==0)].shape[0] / seen_df[(seen_df['random']==1) & (seen_df[y_var]==1)].shape[0]
    class_weight={1:cw}
    scorer = "F1"
    weights_df["sample_weight"] = list(weights_df[y_var+"_sample_weight"].fillna(1).values)
else:
    num_labels = len(cols) 
    weights_df['sample_weight'] = list(weights_df[[x+"_sample_weight" for x in cols]].fillna(1).values)
    seen_df = seen_df.replace(2,1)
    seen_df['labels'] = list(seen_df[cols].values.astype(int))
    seen_df = seen_df.dropna(subset=cols)
    seen_df = seen_df.reset_index(drop=True)
    scorer = "F1 macro"
    class_weight = {}
    for i, t in enumerate(cols):
        cw = seen_df[(seen_df['random']==1) & (seen_df[t]==0)].shape[0] / seen_df[(seen_df['random']==1) & (seen_df[t]==1)].shape[0]
        class_weight[i] = cw

# Remove unneccessary columns
seen_df = seen_df[["id","title","content","labels","random"]+cols].merge(
    weights_df[["doc__id","sample_weight"]].rename(columns={"doc__id":"id"})
)
seen_df.head()

(2513, 105)


,id,title,content,labels,random,INCLUDE,sample_weight
0,3778807,When salient science is not enough to advance ...,Increased social and environmental vulnerabili...,0,0,0.0,1.000000
1,819936,Planning for variable renewable energy and ele...,"High urbanization rates, decentralized solar p...",0,0,0.0,0.666667
2,212843,Modeling climate change and agriculture: an in...,This issue of Agricultural Economics is a spec...,0,0,0.0,1.000000
3,791223,Environmental benefits from ridesharing: A cas...,Emerging ridesharing travel could be an effect...,0,0,0.0,1.000000
4,435782,A viable technology to generate third-generati...,First generation biofuels are widely available...,0,0,0.0,1.000000


In [5]:
# Merge with the unseen data if necessary                                                                
seen_df['seen']  = 1
if args.make_predictions=="True":
    unseen_df = pd.read_csv('../data/0_unlabelled_documents.csv')
    unseen_df['seen'] = 0
    df = (pd.concat([seen_df,unseen_df])
          .sort_values('id')
          .sample(frac=1, random_state=1)
          .reset_index(drop=True)
    )
    df.content = df.content.astype(str)
else:
    df = seen_df
    
nonrandom_index = df[(df['random']!=1) & (df['seen']==1)].index
random_index = df[df['random']==1].index
seen_index = df[df['seen']==1].index
unseen_index = df[df['seen']==0].index



In [6]:
df.shape

(2513, 8)

In [7]:
args.model_name

'climatebert/distilroberta-base-climate-f'

In [ ]:
# import the helper functions defined in cv_setup.py
import cv_setup as cvs

# Get the BERT parameters, and include class_weight as a parameter to be tested
bert_params = cvs.bert_params
bert_params['class_weight'].append(class_weight)
bert_params['class_weight'] = [class_weight]
param_space = list(cvs.product_dict(**bert_params))
params = list(bert_params.keys())

test = False
if test:
    param_space = param_space[:2]

# Get the spilts for our outer fold, discard=False means to pass the nonrandom documents that
# would ordinarily be in the validation set back into the test set
outer_cv = cvs.KFoldRandom(args.n_splits, seen_index, nonrandom_index, discard=False)

# Iterate through the folds
for k, (train, test) in enumerate(outer_cv):    
    print(len(train), len(test))
    cv_results = []
    inner_cv = cvs.KFoldRandom(args.n_splits, train, nonrandom_index, discard=False)
    inner_scores = []
    for l, (l_train, l_test) in enumerate(inner_cv):
        for pr in param_space:
            pr['batch_size'] = 8
            print(pr)
            cv_results.append(cvs.train_eval_bert(
                args.model_name, pr, df=df, targets=cols, train=l_train, test=l_test, 
                roundup=args.roundup
            ))
            break
            w = df.loc[l_test,'sample_weight']

2271 242
{'class_weight': {1: 14.954545454545455}, 'sample_weighted': True, 'batch_size': 8, 'weight_decay': 0, 'learning_rate': 2e-05, 'num_epochs': 4}


Some weights of the model checkpoint at climatebert/distilroberta-base-climate-f were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at climatebert/distilroberta-base-climate-f and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out

could not put model on GPU


/home/max/.local/lib/python3.6/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0
***** Running training *****
  Num examples = 2058
  Num Epochs = 4
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1032


training bert with these params
{'class_weight': {1: 14.954545454545455}, 'sample_weighted': True, 'batch_size': 8, 'weight_decay': 0, 'learning_rate': 2e-05, 'num_epochs': 4}
torch.Size([8, 1]) torch.Size([8])


Step,Training Loss


torch.Size([8, 1]) torch.Size([8])
torch.Size([8, 1]) torch.Size([8])
torch.Size([8, 1]) torch.Size([8])
torch.Size([8, 1]) torch.Size([8])
torch.Size([8, 1]) torch.Size([8])
torch.Size([8, 1]) torch.Size([8])
torch.Size([8, 1]) torch.Size([8])
torch.Size([8, 1]) torch.Size([8])
torch.Size([8, 1]) torch.Size([8])
torch.Size([8, 1]) torch.Size([8])
torch.Size([8, 1]) torch.Size([8])
torch.Size([8, 1]) torch.Size([8])
torch.Size([8, 1]) torch.Size([8])
torch.Size([8, 1]) torch.Size([8])
torch.Size([8, 1]) torch.Size([8])
torch.Size([8, 1]) torch.Size([8])
torch.Size([8, 1]) torch.Size([8])
torch.Size([8, 1]) torch.Size([8])
torch.Size([8, 1]) torch.Size([8])
torch.Size([8, 1]) torch.Size([8])
torch.Size([8, 1]) torch.Size([8])
torch.Size([8, 1]) torch.Size([8])
torch.Size([8, 1]) torch.Size([8])
torch.Size([8, 1]) torch.Size([8])
torch.Size([8, 1]) torch.Size([8])
torch.Size([8, 1]) torch.Size([8])
torch.Size([8, 1]) torch.Size([8])
torch.Size([8, 1]) torch.Size([8])
torch.Size([8, 1]) t

In [17]:
import torch
len(torch.zeros(8).unsqueeze(1).shape)

2

In [ ]:
import itertools

def product_dict(**kwargs):
    keys = kwargs.keys()
    vals = kwargs.values()
    for instance in itertools.product(*vals):
        yield dict(zip(keys, instance))

bert_params = {
  "class_weight": [None],
  "batch_size": [16, 32],
  "weight_decay": (0, 0.3),
  "learning_rate": (1e-5, 5e-5),
  "num_epochs": [2, 3, 4]
}
bert_params['class_weight'].append(class_weight)
params = list(bert_params.keys())
print(params)
param_space = list(product_dict(**bert_params))
scorer = "F1"

In [ ]:
from transformers import AutoTokenizer, BertTokenizer, AutoModel, trainer, BertForSequenceClassification
params = bert_params

model_name = "allenai/scibert_scivocab_uncased"

model = BertForSequenceClassification.from_pretrained(model_name)

In [ ]:
params = param_space[0]
params['class_weight'] = {0: 1, 1: 6.831460674157303}
params

In [ ]:
tokenizer = BertTokenizer.from_pretrained(model_name)

In [ ]:
df = df.head(500)
from cv_setup import create_train_val
train = df[:100]
test = df[100:200]
#df = df.dropna()
#df['labels'] = df.INCLUDE
train_dataset, val_dataset, MAX_LEN = create_train_val(
    tokenizer, df['content'].astype(str), 
    df['labels'], 
    df.index[0:100], 
    df.index[100:200], 
    tensorflow=False
)

In [ ]:
train_dataset

In [ ]:
val_dataset

In [ ]:
import torch

train_index = df.index[0:10]
test_index = df.index[100:200]

x = df['content']
y = df['labels']

train_encodings= tokenizer(list(x[train_index]),truncation=True,padding=True,max_length=512)
val_encodings = tokenizer(list(x[test_index]),truncation=True,padding=True,max_length=512)

class TDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx],dtype=torch.float32)
        return item

    def __len__(self):
        return len(self.labels)
    
train_dataset = TDataset(train_encodings, list(y[train_index]))
val_dataset = TDataset(val_encodings, list(y[test_index]))


In [ ]:
import torch 
torch.zeros([16]).shape

In [ ]:
model.config

In [ ]:
torch.__version__


In [ ]:
from transformers import Trainer, TrainingArguments

params = param_space[1]
params['batch_size'] = 4
params['num_epochs'] = 1
params['class_weight'] = class_weight

model = BertForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

from transformers import Trainer, TrainingArguments

class CWTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        try:
            device = torch.device('cuda:0')
        except:
            print("no device")
        labels = labels.to(device)
        
        if "sample_weight" in inputs.keys():
            sample_weight = inputs.pop("sample_weight")
        else:
            sample_weight = None
        outputs = model(**inputs)
        logits = outputs.logits

        if self.class_weight is not None:
            cw = torch.tensor(list(self.class_weight.values()))

        else:
            cw = None
        loss_fct = torch.nn.BCEWithLogitsLoss(pos_weight=cw,reduction='none')
        loss = loss_fct(logits,labels.float())
        #loss = loss_fct(logits.view(-1, model.num_labels),
        #                labels.float().view(-1, model.num_labels))
        if sample_weight is not None:
            loss = (loss * sample_weight / sample_weight.sum()).sum().mean()
        else:
            loss = loss.mean()
        return (loss, outputs) if return_outputs else loss




class BCWTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        cw = torch.tensor(self.class_weight[1])
        loss_fct = torch.nn.BCEWithLogitsLoss(pos_weight=cw)
        loss = loss_fct(logits.view(-1, 1),
                        labels.float().view(-1, 1))
        return (loss, outputs) if return_outputs else loss



training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=params['num_epochs'],
    per_device_train_batch_size=params['batch_size'],
    learning_rate=params['learning_rate'],
    weight_decay=params['weight_decay'],
    gradient_checkpointing=True
)

trainer = CWTrainer(
    model=model,
    args = training_args, 

    #optimizers=[optimizer]
)
trainer.class_weight = params['class_weight']
#trainer.class_weight=torch.tensor(list(params["class_weight"].values()))
trainer.train_dataset = train_dataset
trainer.train()

In [31]:
preds = trainer.predict(val_dataset)
preds

***** Running Prediction *****
  Num examples = 100
  Batch size = 8


5


5
5
5
5
5
5
5
5
5
5
5
5


PredictionOutput(predictions=array([[-8.92750919e-04, -3.03136647e-01, -1.38864815e-01,
         6.98384702e-01,  3.95447195e-01],
       [ 1.57699585e-01, -5.91032207e-01, -2.06474334e-01,
         5.79557359e-01,  4.95105952e-01],
       [-1.48357376e-02, -5.85529447e-01,  4.19373810e-02,
         7.41156518e-01,  4.59309220e-01],
       [-1.32918879e-01, -6.13522530e-01, -3.00120413e-02,
         5.59860528e-01,  6.39672160e-01],
       [-2.53252238e-02, -2.24090785e-01, -1.12039998e-01,
         9.25198257e-01,  1.25383645e-01],
       [ 6.51370734e-03, -7.38315657e-02, -2.66591877e-01,
         9.57293272e-01,  2.33598292e-01],
       [-8.65197778e-02, -5.30148506e-01, -2.62979597e-01,
         1.84291363e-01,  3.85540426e-01],
       [ 1.28535986e-01, -1.30125344e-01, -2.80426085e-01,
         3.67568821e-01,  3.64873350e-01],
       [ 3.82994860e-02, -1.16870873e-01, -2.80790508e-01,
         1.02078664e+00,  1.74427435e-01],
       [-5.51003888e-02, -3.46435606e-01, -2.93871760

In [31]:

activation = torch.nn.Sigmoid()
activation(torch.tensor(preds.predictions)).numpy()

array([[0.5596566 , 0.5017008 , 0.44246215, 0.553499  , 0.5168284 ],
       [0.5560532 , 0.5339529 , 0.3784678 , 0.5812366 , 0.4403819 ],
       [0.4711132 , 0.5828854 , 0.45413065, 0.49687284, 0.45692253],
       [0.5299767 , 0.46306592, 0.42286575, 0.55005836, 0.4875898 ],
       [0.4848466 , 0.5505157 , 0.43499762, 0.53591657, 0.43466163],
       [0.5593149 , 0.558594  , 0.45497262, 0.58731556, 0.48736408],
       [0.56025034, 0.44336146, 0.42529953, 0.5171849 , 0.4468247 ],
       [0.5331194 , 0.5653038 , 0.43823856, 0.55371565, 0.4658319 ],
       [0.47057995, 0.51654625, 0.40125751, 0.5644671 , 0.49133053],
       [0.50247   , 0.64521986, 0.5430379 , 0.54836124, 0.49559474],
       [0.5702121 , 0.5672348 , 0.44686088, 0.50926226, 0.45129716],
       [0.43598104, 0.55442256, 0.3894622 , 0.559431  , 0.45803875],
       [0.5600321 , 0.5168124 , 0.42000622, 0.5372617 , 0.48540455],
       [0.47850484, 0.6429855 , 0.4236073 , 0.4823718 , 0.44471532],
       [0.4660152 , 0.5744736 , 0.

In [ ]:
#cv_df.loc[pd.notna(cv_df['class_weight']),'class_weight'] = ast.literal_eval(cv_df.loc[pd.notna(cv_df['class_weight']),'class_weight'])



cv_df.loc[pd.notna(cv_df['class_weight']),'class_weight'] = cv_df.loc[pd.notna(cv_df['class_weight']),'class_weight'].apply(lambda x: ast.literal_eval(x))

cv_df.loc[pd.notna(cv_df['class_weight']),'class_weight'] = cv_df.loc[pd.notna(cv_df['class_weight']),'class_weight'].astype(str)
best_model = (cv_df[pd.notna(cv_df[scorer])]
              .groupby(params)[scorer]
              .mean()
              .sort_values(ascending=False)
              .reset_index()
             ).to_dict('records')[0]

best_model

In [8]:
pd.__version__

'1.1.5'

In [9]:
pr = param_space[0]
import ast
params_tested=pd.read_csv(fname)[list(pr.keys())].to_dict('records')
for pr in params_tested:
    if pd.isna(pr["class_weight"]):
        pr["class_weight"] = None
    if isinstance(pr['class_weight'],str):
        pr["class_weight"] = ast.literal_eval(pr["class_weight"])
        
for pr in param_space:
    pr['weight_decay']
    if pr in params_tested:
        continue
    else:
        print(pr)
        #break


{'class_weight': {0: 1, 1: 6.831460674157303}, 'batch_size': 16, 'weight_decay': 0, 'learning_rate': 5e-05, 'num_epochs': 4}
{'class_weight': {0: 1, 1: 6.831460674157303}, 'batch_size': 16, 'weight_decay': 0.3, 'learning_rate': 1e-05, 'num_epochs': 2}
{'class_weight': {0: 1, 1: 6.831460674157303}, 'batch_size': 16, 'weight_decay': 0.3, 'learning_rate': 1e-05, 'num_epochs': 3}
{'class_weight': {0: 1, 1: 6.831460674157303}, 'batch_size': 16, 'weight_decay': 0.3, 'learning_rate': 1e-05, 'num_epochs': 4}
{'class_weight': {0: 1, 1: 6.831460674157303}, 'batch_size': 16, 'weight_decay': 0.3, 'learning_rate': 5e-05, 'num_epochs': 2}
{'class_weight': {0: 1, 1: 6.831460674157303}, 'batch_size': 16, 'weight_decay': 0.3, 'learning_rate': 5e-05, 'num_epochs': 3}
{'class_weight': {0: 1, 1: 6.831460674157303}, 'batch_size': 16, 'weight_decay': 0.3, 'learning_rate': 5e-05, 'num_epochs': 4}
{'class_weight': {0: 1, 1: 6.831460674157303}, 'batch_size': 32, 'weight_decay': 0, 'learning_rate': 1e-05, 'num_

In [8]:
pr['weight_decay'] = 0
pr in params_tested

False

In [9]:
x = {}
x["weight_decay"] = 1

params = [{"weight_decay":1.0},]
x in params

True

In [10]:
[x for x in params_tested if x['class_weight']!=None and x['batch_size']==32]

[{'class_weight': {0: 1, 1: 6.831460674157303},
  'batch_size': 32,
  'weight_decay': 0.0,
  'learning_rate': 1e-05,
  'num_epochs': 2},
 {'class_weight': {0: 1, 1: 6.831460674157303},
  'batch_size': 32,
  'weight_decay': 0.0,
  'learning_rate': 1e-05,
  'num_epochs': 3},
 {'class_weight': {0: 1, 1: 6.831460674157303},
  'batch_size': 32,
  'weight_decay': 0.0,
  'learning_rate': 1e-05,
  'num_epochs': 4},
 {'class_weight': {0: 1, 1: 6.831460674157303},
  'batch_size': 32,
  'weight_decay': 0.0,
  'learning_rate': 1e-05,
  'num_epochs': 2},
 {'class_weight': {0: 1, 1: 6.831460674157303},
  'batch_size': 32,
  'weight_decay': 0.0,
  'learning_rate': 1e-05,
  'num_epochs': 3},
 {'class_weight': {0: 1, 1: 6.831460674157303},
  'batch_size': 32,
  'weight_decay': 0.0,
  'learning_rate': 1e-05,
  'num_epochs': 4},
 {'class_weight': {0: 1, 1: 6.831460674157303},
  'batch_size': 32,
  'weight_decay': 0.0,
  'learning_rate': 1e-05,
  'num_epochs': 2},
 {'class_weight': {0: 1, 1: 6.83146067415

In [25]:
best_model = (cv_df
              .groupby(params)[scorer]
              .mean()
              .sort_values(ascending=False)
              .reset_index()
             ).to_dict('records')[0]


In [18]:
cv_df

,ROC AUC,F1,precision,recall,accuracy,class_weight,batch_size,weight_decay,learning_rate,num_epochs,dataset_size
0,0.808260,0.818692,0.782143,0.858824,0.755051,NaN,16,0.0,0.00001,2,1960
1,0.803769,0.813869,0.761092,0.874510,0.742424,NaN,16,0.0,0.00001,3,1960
2,0.791893,0.799228,0.787072,0.811765,0.737374,NaN,16,0.0,0.00001,4,1960
3,0.807064,0.808587,0.743421,0.886275,0.729798,NaN,16,0.0,0.00005,2,1960
4,0.791573,0.802326,0.793103,0.811765,0.742424,NaN,16,0.0,0.00005,3,1960
...,...,...,...,...,...,...,...,...,...,...,...
86,0.500000,0.000000,0.000000,0.000000,0.356061,"{0: 1, 1: 6.831460674157303}",16,0.3,0.00005,3,1960
87,0.500000,0.000000,0.000000,0.000000,0.356061,"{0: 1, 1: 6.831460674157303}",16,0.3,0.00005,4,1960
88,0.735002,0.788253,0.650510,1.000000,0.654040,"{0: 1, 1: 6.831460674157303}",32,0.0,0.00001,2,1960
89,0.792032,0.791667,0.669377,0.968627,0.671717,"{0: 1, 1: 6.831460674157303}",32,0.0,0.00001,3,1960


In [20]:
params = list(bert_params.keys())

In [22]:
best_model = (cv_df
              .groupby(params)[scorer]
              .mean()
              .sort_values(ascending=False)
              .reset_index()
             )
best_model

,class_weight,batch_size,weight_decay,learning_rate,num_epochs,F1
0,"{0: 1, 1: 6.831460674157303}",16,0.0,0.00001,4,0.802815
1,"{0: 1, 1: 6.831460674157303}",16,0.0,0.00001,3,0.795534
2,"{0: 1, 1: 6.831460674157303}",16,0.0,0.00005,2,0.794413
3,"{0: 1, 1: 6.831460674157303}",32,0.0,0.00001,4,0.793147
4,"{0: 1, 1: 6.831460674157303}",16,0.0,0.00005,4,0.792043
5,"{0: 1, 1: 6.831460674157303}",16,0.0,0.00005,3,0.791254
6,"{0: 1, 1: 6.831460674157303}",32,0.0,0.00001,3,0.790273
7,"{0: 1, 1: 6.831460674157303}",32,0.0,0.00001,2,0.788962
8,"{0: 1, 1: 6.831460674157303}",16,0.0,0.00001,2,0.788099
9,"{0: 1, 1: 6.831460674157303}",16,0.3,0.00005,4,0.000000
